In [1]:
%matplotlib inline
import numpy as np
import pylab as pl
import time
import scipy.special as special
from scipy.integrate import quad, dblquad, fixed_quad

# Set plot parameters to make beautiful plots
pl.rcParams['figure.figsize']  = 12, 7.5
pl.rcParams['lines.linewidth'] = 1.5
pl.rcParams['font.family']     = 'serif'
pl.rcParams['font.weight']     = 'bold'
pl.rcParams['font.size']       = 20  
pl.rcParams['font.sans-serif'] = 'serif'
pl.rcParams['text.usetex']     = True
pl.rcParams['axes.linewidth']  = 1.5
pl.rcParams['axes.titlesize']  = 'medium'
pl.rcParams['axes.labelsize']  = 'large'

pl.rcParams['xtick.major.size'] = 8     
pl.rcParams['xtick.minor.size'] = 4     
pl.rcParams['xtick.major.pad']  = 8     
pl.rcParams['xtick.minor.pad']  = 8     
pl.rcParams['xtick.color']      = 'k'     
pl.rcParams['xtick.labelsize']  = 'large'
pl.rcParams['xtick.direction']  = 'in'    

pl.rcParams['ytick.major.size'] = 8     
pl.rcParams['ytick.minor.size'] = 4     
pl.rcParams['ytick.major.pad']  = 8     
pl.rcParams['ytick.minor.pad']  = 8     
pl.rcParams['ytick.color']      = 'k'     
pl.rcParams['ytick.labelsize']  = 'large'
pl.rcParams['ytick.direction']  = 'in'

In [2]:
#testing I_star(0)

def I_star_integrand(x, alpha, delta):
    ans = np.cos(alpha * x) * special.j0(delta * np.sqrt(1. - x**2.))
    return ans

def I_star_analytic(alpha, delta):
    A     = np.sqrt(alpha**2. + delta**2.)
    term1 = np.sin(alpha) / alpha
    num   = -A * np.sin(alpha) + alpha * np.sin(A)
    denom = A * np.sqrt(A - alpha) * np.sqrt(A + alpha)
    term2 = delta / alpha * num / denom
    ans   = term1 + term2
    return ans

alpha = 1.1231
delta = 10.12312

print quad(lambda x: I_star_integrand(x, alpha, delta), 0., 1.)[0]

print I_star_analytic(alpha, delta)

-0.0676714314003
-0.0676714314003


In [3]:
#testing I_3(0)

def I_3_integrand(x, alpha, delta):
    ans = x**2. * np.cos(alpha * x) * special.j0(delta * np.sqrt(1. - x**2.))
    return ans

def I_3_analytic(alpha, delta):
    A     = np.sqrt(alpha**2. + delta**2.)
    plus  = np.sqrt(A + alpha)
    minus = np.sqrt(A - alpha)
    num   = (2. * alpha * A**5. * (delta - minus * plus) * np.cos(alpha)
             - alpha**3. * delta * (2. * alpha**2. - delta**2.) * A * np.cos(A)
             - 2. * alpha**4. * delta * A * np.sin(alpha)
             + alpha**6. * delta * A * np.sin(alpha)
             - 4. * alpha**2. * delta**3. * A * np.sin(alpha)
             + 2. * alpha**4. * delta**3. * A * np.sin(alpha)
             - 2. * delta**5. * A * np.sin(alpha)
             + alpha**2. * delta**5. * A * np.sin(alpha)
             + 2. * alpha**4. * A * minus * plus * np.sin(alpha)
             - alpha**6. * A * minus * plus * np.sin(alpha)
             + 4. * alpha**2. * delta**2. * A * minus * plus * np.sin(alpha)
             - 2. * alpha**4. * delta**2. * A * minus * plus * np.sin(alpha)
             + 2. * delta**4. * A * minus * plus * np.sin(alpha)
             - alpha**2. * delta**4. * A * minus * plus * np.sin(alpha)
             + 2. * alpha**5. * delta * np.sin(A)
             - alpha**7. * delta * np.sin(A)
             - alpha**3. * delta**3. * np.sin(A)
             - alpha**5. * delta**3. * np.sin(A)
    )
    denom = alpha**3. * A**5. * minus**7. * plus**7.
    ans = delta**6. * num / denom
    return ans * -1. #why am I off by a factor of -1?

print quad(lambda x: I_3_integrand(x, alpha, delta), 0., 1.)[0]
print I_3_analytic(alpha, delta)

0.00527801500735
0.00527801500735


In [4]:
#testing I_star(1)

def I_star_integrand1(x, alpha, delta):
    ans = np.cos(alpha * x) * special.jn(1, delta * np.sqrt(1. - x**2.))
    return ans

def I_star_analytic1(alpha, delta):
    plus  = np.sqrt(np.sqrt(alpha**2. + delta**2.) + alpha)
    minus = np.sqrt(np.sqrt(alpha**2. + delta**2.) - alpha)
    term1 = delta * np.pi / 4.
    num   = (special.j0(1./2. * plus**2.) * special.jn(1, 1./2. * minus**2.)
            + special.j0(1./2. * minus**2.) * special.jn(1, 1./2. * plus**2.))
    denom = np.sqrt(alpha**2. + delta**2.)
    ans   = term1 * num / denom
    return ans

print quad(lambda x: I_star_integrand1(x, alpha, delta), 0., 1.)[0]
print I_star_analytic1(alpha, delta)

0.0720965444146
0.0720965444146


In [5]:
#calculate first derivative of I_star(1)
def Istar1_firstderiv(alpha, delta):
    A          = np.sqrt(alpha**2. + delta**2.)
    plus       = A + alpha
    minus      = A - alpha
    prefactor  = delta * np.pi / 4. * 1./A**3.
    term1      = special.j0(1./2. * minus) * (alpha * A * special.j0(1./2. * plus) 
                                             - plus * special.jn(1, 1./2. * plus))
    term2num   = special.jn(1, 1./2. * minus) * (delta**2. * special.j0(1./2. * plus) 
                                               - alpha * (delta**2. + alpha * plus) * special.jn(1, 1./2. * plus))
    term2denom = plus
    ans        = prefactor * (term1 + term2num/term2denom)
    return ans

def Istar1_firstderiv_integrand(x, alpha, delta):
    ans = - x * np.sin(alpha * x) * special.jn(1, delta * np.sqrt(1. - x**2.))
    return ans

print quad(lambda x: Istar1_firstderiv_integrand(x, alpha, delta), 0., 1.)[0]
print Istar1_firstderiv(alpha, delta)

-0.0174996504633
-0.0174996504633


In [6]:
#calculate third derivative of I_star(1)
def Istar1_thirdderiv(alpha, delta):
    A          = np.sqrt(alpha**2. + delta**2.)
    plus       = A + alpha
    minus      = A - alpha
    prefactor  = np.pi / 4. * delta * 1./A**7.
    inner      = (special.jn(1, -minus/2.) * ((12. * alpha**3. - 3. * alpha**5. 
                                             + 3. * alpha * delta**2. * (-1. + delta**2.)
                                             + 2. * alpha**4. * A
                                             + 3. * alpha**2. * (-4. + delta**2.) * A
                                             + delta**2. * (3. + delta**2.) * A) * special.j0(plus/2.)
                                             - alpha * A * (alpha**4. + 13. * delta**2. 
                                                            + alpha**2. * (-2. + delta**2.))
                                            * special.jn(1, plus/2.))
                + special.j0(-minus/2.) * (-alpha * A * (alpha**4. + 3. * delta**2.
                                       + alpha**2. * (-12. + delta**2.)) * special.j0(plus/2.)
                                       + (-12. * alpha**3. + 3. * alpha**5. 
                                          - 3. * alpha * delta**2. * (-1. + delta**2.)
                                          + 2. * alpha**4. * A + 3. * alpha**2. * (-4. + delta**2.) * A
                                          + delta**2. * (3. + delta**2.) * A) * special.jn(1, plus/2.)))
    ans         = prefactor * inner
    return ans


def Istar1_thirdderiv_integrand(x, alpha, delta):
    ans = x**3. * np.sin(alpha * x) * special.jn(1, delta * np.sqrt(1. - x**2.))
    return ans

print quad(lambda x: Istar1_thirdderiv_integrand(x, alpha, delta), 0., 1.)[0]
print Istar1_thirdderiv(alpha, delta)

0.0077745698208
0.00777456982085


In [15]:
#retrying I_2 calculation

alpha = 1.5
delta = -1. #NOTE: ANSWER IS OFF BY FACTOR OF -1 WHEN DELTA IS NEGATIVE

def I_2_integrand(x, alpha, delta):
    ans = x * np.sqrt(1. - x**2.) * np.sin(1 * alpha * x) * special.jn(1, delta * np.sqrt(1. - x**2.))
    return ans

def I_2_analytic(alpha, delta):
    A     = np.sqrt(alpha**2. + delta**2.)
    plus  = np.sqrt(A + alpha)
    minus = np.sqrt(A - alpha)
    num   = alpha * delta * (3. * A * np.cos(A) + (-3. + alpha**2. + delta**2.) * np.sin(A))
    denom = A**5.
    ans   = 2. * num / denom
    return -ans


print quad(lambda x: I_2_integrand(x, alpha, delta), -1., 1.)[0]
print I_2_analytic(alpha, delta)

-0.157562811158
-0.157562811158


In [11]:
#calculating I_1(0)

alpha = 3.1231
delta = -12.5 

def I_1_integrand(x, alpha, delta):
    ans = (1. - x**2.) * np.cos(alpha * x) * special.j0(delta * np.sqrt(1. - x**2.))
    return ans

def I_1_analytic(alpha, delta):
    A     = np.sqrt(alpha**2. + delta**2.)
    minus = np.sqrt(A - alpha)
    plus  = np.sqrt(A + alpha)
    ds    = -np.sign(delta)
    outer = 2. * delta**6. / (alpha**3. * A**5. * minus**7. * plus**7.)
    inner = ((ds*2. * alpha * A**5. * (delta + ds*minus * plus)) * np.cos(alpha)
             + alpha**3. * delta * ds*(2. * alpha**2. - delta**2.) * A * np.cos(A)
             + ds*2. * alpha**4. * delta * A * np.sin(alpha)
             + ds*4. * alpha**2. * delta**3. * A * np.sin(alpha)
             + ds*2. * delta**5. * A * np.sin(alpha)
             + 2. * alpha**4. * A * minus * plus * np.sin(alpha)
             + 4. * alpha**2. * delta**2. * A * minus * plus * np.sin(alpha)
             + 2. * delta**4. * A * minus * plus * np.sin(alpha)
             - ds*alpha**3. * delta * (2. * alpha**2. + (-1. + alpha**2.) * delta**2. + delta**4.) * np.sin(A))
    ans   = outer * inner
    return ans

def I_1_shortened(alpha, delta):
    A     = np.sqrt(alpha**2. + delta**2.)
    minus = np.sqrt(A - alpha)
    plus  = np.sqrt(A + alpha)
    if(alpha == 0 or delta == 0 or minus == 0.):
        return 0.
    ans = 2. * ( (2. * alpha**2. + (alpha**2. - 1.)*delta**2. + delta**4.)*np.sin(A) 
                - (2. * alpha**2. - delta**2.) * A * np.cos(A)) / A**5.
    return ans


print quad(lambda x: I_1_integrand(x, alpha, delta), -1., 1.)[0]
print I_1_shortened(alpha, delta)

0.0548519338781
0.0548519338781
